### Лабораторная работа №1. Симплекс-метод
Выполнили студенты: Петренко Людмила М33001, Кусайкина Елизавета М33001, Шалимов Иван М33021

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json

**Задача линейного программирования** - оптимизация линейной функции с некоторым набором ограничений. 

**Каноническая форма** задачи линейного программирования предполагает, что все ограничения будут заданы равенствами, а все переменные будут неотрицательными. Для приведения к канонической форме используется добавление балансирующих переменных - увеличение размерности.

##### Алгоритм Симплекс-метода.

1. Построение опорного плана
    * После того, как задача была приведена к канонической форме, среди переменных функции выбирают базисные. В большинстве случаев в качестве базисных будут использоваться балансирующие переменные.

    * **Опорный план** - начальные значения переменных, одна из точек многогранника решений. В большинстве случаев мы заполняем небазисные переменные нулями, а значения для базисных берем из столбца свободных членов системы.
    * Необходимо построить симплекс-таблицу, соответствующую выбранному опорному плану.

2. Построение оптимального плана
    * Для того, чтобы опорный план стал оптимальным, с помощью симплекс-таблицы ищут разрешающие строку и столбец, а затем меняют базисную переменную из строки и небазисную из столбца местами, пересчитывая матрицу системы, симплекс-таблицу и соответственно опорный план.
    * Разрешающие строка и столбец выбираются таким образом, чтобы минимизировать (или максимизировать, зависит от условий задачи) значение целевой функции для нового опорного плана. 

        * При минимизации выбирают разрешающий столбец, который имеет минимальный отрицательный коэффициент в функции. Тогда при его увеличении, значение функции уменьшится.
        * При максимизации, соответственно, выбирают столбец с максимальным положительным коэффициентом, чтобы увеличивая значение переменной, увеличить целевую функцию.

    * Следовательно, опорный план становится оптимальным, когда либо все коэффициенты функции неотрицательны - в случае минимизации, либо все коэффициенты неположительны - в случае максимизации.

    * Выбор разрешающей строки делается так, чтобы для выбранной переменной, отношение соответствующего ей свободного члена к значению из разрешающего столбца было минимальным.

Класс **Linear task** описывает задачу линейного программирования: коэффициенты оптимизируемой линейной функции и ограничения, а также предоставляет реализацию симплекс метода для решения задачи.

Данные о функции и ограничения считываются из json файла, в качестве результата работы симплекс метода предоставляется вектор значений аргументов.

In [2]:
class Linear_task:
    def __init__(self):
        self.func: np.array(int)
        self.goal: str
        self.constraints: list[dict()]
        self.constraints_matrix: np.array()
        self.b_array: np.array()
        self.original_size: int

        self.simplex_table = dict()
        self.optimal: np.array()
        self.basis: np.array()
        self.func_b = 0.0
    
    # считывание условий задачи из файла json
    def load_data_from_json(self, filename: str) -> None:
        with open(filename) as f:
            data = json.load(f)
        self.func = np.array(data[' f '].copy()).astype(np.float32)
        self.original_size = len(self.func)
        self.goal = data[' goal ']
        self.constraints = data[' constraints ']
        self.constraints_matrix = np.array([line[' coefs '] for line in self.constraints]).astype(np.float32)
        self.b_array = np.array([line[' b '] for line in self.constraints]).astype(np.float32)
        self.make_canonic_form()

    # переход к канонической форме задачи - добавление балансирующих переменных
    def make_canonic_form(self) -> None:
        for index, line in enumerate(self.constraints):
            if line[' type '] != ' eq ':
                if line[' type '] == ' gte ':
                    for item in self.constraints_matrix[index]:
                        item = -item
                self.constraints_matrix = np.append(self.constraints_matrix, 
                        [[1] if i == index else [0] for i in range(len(self.constraints_matrix))], axis= 1)
                self.func = np.append(self.func, 0)
        
    def simplex_method(self):
        self.set_start_values()

        while True:
            column = self.choose_solving_column()
            # остановка алгоритма
            if column == -1:
                break
            try:
                row, new_value = self.choose_solving_row(column)
            except ValueError:
                return "Система ограничений несовместна"
            self.recount_matrix(column, row)
            self.recount_optimal(column, new_value, row)
            self.recount_simplex_table(column, row)
            self.change_func_coefs(column, row)
            print("Значение функции:    ", np.dot(self.optimal, self.func.T) + self.func_b)

        return self.optimal[: self.original_size]

    def change_func_coefs(self, index_new_basis, index_old_basis):

        for i in range(len(self.constraints_matrix)):
            if self.constraints_matrix[i][index_new_basis] != 0:
                self.func_b += self.func[index_new_basis]*self.b_array[i]
                for j, value in enumerate(self.constraints_matrix[i]):
                    if j != index_new_basis:
                        self.func[j] += -self.func[index_new_basis]*value
                        
        self.func[index_new_basis] = 0
            
    def recount_simplex_table(self, index_new_basis, index_old_basis):
        self.simplex_table[index_new_basis] = dict.fromkeys(list(self.simplex_table[index_old_basis].keys()))
        del self.simplex_table[index_old_basis]
        
        for key, value in self.simplex_table.items():
            del value[index_new_basis]
            for i in range(len(self.constraints_matrix)):
                if self.constraints_matrix[i][key] != 0:
                    value[index_old_basis] = self.constraints_matrix[i][index_old_basis]/self.constraints_matrix[i][key]
        
        for key_ in self.simplex_table.keys():
            for key, value in self.simplex_table[key_].items():
                if key == index_old_basis:
                    continue
                for i in range(len(self.constraints_matrix)):
                    if self.constraints_matrix[i][key_] == 0:
                        continue
                    self.simplex_table[key_][key] = self.constraints_matrix[i][key]/self.constraints_matrix[i][key_]
    
    def recount_matrix(self, old_index, new_index):
        matrix = np.copy(self.constraints_matrix)
        basis_indices = list(self.simplex_table.keys())
        basis_indices.remove(new_index)
        b = np.copy(self.b_array)
        line_number = -1
        for row in range(len(self.constraints_matrix)):
            flag_right_line = True
            for k in basis_indices:
                if matrix[row][k] != 0:
                    flag_right_line = False
            if matrix[row][old_index] == 0:
                continue
            elif flag_right_line:
                line_number = row
                break

        for row in range(len(self.constraints_matrix)):
            if matrix[row][old_index] == 0:
                continue
            elif row == line_number:
                f = matrix[row][old_index]
                for j in range(len(self.constraints_matrix[row])):
                    matrix[row][j] /= f
                b[row] /= f
            else:
                f = matrix[row][old_index] / matrix[line_number][old_index]
                for j in range(len(self.constraints_matrix[row])):
                    matrix[row][j] -= f * matrix[line_number][j]
                b[row] -= f * b[line_number]
            
        self.constraints_matrix = matrix
        self.b_array = b

    def recount_optimal(self, index_new_basis, new_value, index_old_basis):
        for i in self.simplex_table.keys():
            for row in range(len(self.constraints_matrix)):
                if self.constraints_matrix[row][i] != 0:
                    self.optimal[i] = self.b_array[row]/self.constraints_matrix[row][i]

        self.optimal[index_new_basis] = new_value
        self.optimal[index_old_basis] = 0

    def set_start_values(self):
        self.optimal = [0 for i in range(len(self.func))]
        # определение базисных векторов
        self.simplex_table = dict.fromkeys([i for i in range(self.original_size, len(self.func))])

        for k in range(len(self.constraints_matrix)):
            for i in range(self.original_size):
                if self.constraints_matrix[k][i] == 0:
                    continue
                flag_basis = True
                for j in self.simplex_table.keys():
                    if self.constraints_matrix[k][j] != 0:
                        flag_basis = False
            if flag_basis:
                self.simplex_table[i] = None
        nonbasis = [i for i in range(len(self.func)) if i not in self.simplex_table]

        for i in range(len(self.func)):
            if i in self.simplex_table:
                for k in range(len(self.constraints_matrix)):
                    if self.constraints_matrix[k][i] == 0:
                        continue
                    row = {j : self.constraints_matrix[k][j]/self.constraints_matrix[k][i] for j in nonbasis}
                    self.simplex_table[i] = row.copy()
                    self.optimal[i] = self.b_array[k]
            else:
                self.optimal[i] = 0

        self.optimal = np.array(self.optimal)
        self.optimal_func = np.dot(self.optimal, self.func.T)

    def choose_solving_row(self, column):
        dict_ = {key: abs(self.optimal[key]/value[column]) for key, value in self.simplex_table.items()
                  if value[column] != 0 and self.optimal[key]/value[column] > 0}
        if len(list(dict_.values())) == 0:
            raise ValueError
        min_value = min(list(dict_.values()))
        row = list(dict_.keys())[list(dict_.values()).index(min_value)]

        return row, min_value

    def choose_solving_column(self):
        max_index = -1
        min_index = -1
        for i in list(self.simplex_table[list(self.simplex_table.keys())[0]].keys()):
            if self.goal == " max ":
                if self.func[i] >= self.func[max_index] and self.func[i] >= 0:
                    max_index = i
            elif self.goal == " min ":
                if self.func[i] < self.func[min_index] and self.func[i] <= 0:
                    min_index = i
        
        if self.goal == " max ":
            return max_index
        elif self.goal == " min ":
            return min_index

#### Примеры результатов.

##### 1. Максимизация функции с равенством

$f(x) = x_1 + 2x_2 + 3x_3$ -> max

$\begin{cases}
x_1 \le 1  \\
x_1 + x_2 \ge 2   \\
x_1 + x_2 + x_3 = 3 
\end{cases}$

In [152]:
task = Linear_task()
task.load_data_from_json('data.json')
print("Решение: ", task.simplex_method())

Значение функции:     7.0
Решение:  [0. 2. 1.]


##### 2. Максимизация функции с неравенствами

$f(x) = 2x_1 + 3x_2$ -> max

$\begin{cases}
x_1 \le 40  \\
x_2 \le 30  \\
x_1 + x_2 \le 60   \\
x_1 + 2x_2 \le 80 
\end{cases}$

In [153]:
task = Linear_task()
task.load_data_from_json('data_2.json')
print("Решение: ", task.simplex_method())

Значение функции:     90.0
Значение функции:     130.0
Значение функции:     140.0
Решение:  [40. 20.]


##### 3. Минимизация функции

$f(x) = x_1 - x_2$ -> min

$\begin{cases}
x_1 - 2x_2 \le 1  \\
2x_1 - x_2 \ge 2  \\
3x_1 + x_2 \le 3 
\end{cases}$

In [154]:
task = Linear_task()
task.load_data_from_json('data_3.json')
print("Решение: ", task.simplex_method())

Значение функции:     -2.0
Значение функции:     -2.2000000029802322
Решение:  [0.2       2.4000001]


##### 4. Минимизация функции, функция неограничена снизу

$f(x) = -x_1 - x_2$ -> min

$\begin{cases}
-x_1 + x_2 \le 1  \\
x_1 - 2x_2 \le 2 
\end{cases}$

In [155]:
task = Linear_task()
task.load_data_from_json('data_2_error.json')
print("Решение: ", task.simplex_method())

Значение функции:     -2.0
Решение:  Система ограничений несовместна


#### Тестирование

In [5]:
task = Linear_task()
task.load_data_from_json('data_1.json')
print("Решение: ", task.simplex_method())

Значение функции:     13.333333730697632
Значение функции:     17.066666841506958
Значение функции:     18.65853664080302
Решение:  [2.17073156 1.21951234 1.51219499]
